In [2]:
from sqlalchemy import create_engine
import pandas as pd

from config import db_connection_string

In [3]:
engine = create_engine(db_connection_string)

## match contribution and tx

In [4]:
df = pd.read_sql_query("""
SELECT 
	`t1`.`id` AS `contribution`,
	`t1`.`account`,
    `t1`.`grant`,
	`t3`.`from`,
	`t3`.`tx`
FROM
    `contribution` AS `t1`
INNER JOIN
    `grant` AS `t2` 
	ON 
		`t2`.`id` = `t1`.`grant`
INNER JOIN
    `tx` AS `t3` 
    ON 
		`t3`.`to` = `t2`.`address`
        AND `t3`.`symbol` = `t1`.`symbol`
        AND `t3`.`amount` = `t1`.`amount_to_grant`
        AND ABS(TIMESTAMPDIFF(MINUTE, `t3`.`created_on`, `t1`.`created_on`)) <= 5
""", engine)

In [5]:
df.shape, df['tx'].nunique()

((232184, 5), 162853)

In [6]:
df['n_account_in_tx'] = df.groupby('tx')['account'].transform('nunique')
df['contribution_exists_n_times'] = df.groupby('contribution')['contribution'].transform(len)
df['tx_exists_n_times_in_grant'] = df.groupby(['grant', 'tx'])['tx'].transform(len)


## tx (only matching one account)

In [48]:
account_address_df = df[df['n_account_in_tx']==1].groupby(['account', 'from']).size().reset_index(name='tx_count')

In [56]:
engine.execute("""
    CREATE TABLE `account_address`(
        `account` VARCHAR(300) NOT NULL,
        `address` VARCHAR(256) NOT NULL,
        `tx_count` INT NOT NULL,
        KEY `account_index`(`account`),
        KEY `address_index`(`address`)
    )
""")

In [58]:
account_address_df.rename(columns={'from':'address'}).to_sql("account_address", engine, index=False, if_exists="append")

## other interesting things

### addresses used by multiple accounts

In [49]:
account_address_df['from'].value_counts()

0x18a9e43b760e78af0d9f5b259d4ed2b0e27330f6    10
0x130c96c7f196654517158d06f6d66354a9003b53     9
0x8cce3eb41e2f22557514d291a91d3b80bc41a722     5
0x1584225583e707e2b1d61270a6aa249d57db9f9a     4
0x9078aee592d97a7de8c9df3a0ebb938ed88ab97e     4
                                              ..
0x4818ff9b82dc202ea44d6497ec7aca644a5bdaf2     1
0x29220a6cb58e3862c28f01741117f63b66ff1f06     1
0xd80e912e05c61711466f9bbe8ef656ff0d5ac83a     1
0x2e44eabb2901fefa4600994bca2b80cd83cf3178     1
0x222c0ca3169a8e2111e62f573900bdfe75a79894     1
Name: from, Length: 17652, dtype: int64

In [54]:
(account_address_df['from'].value_counts()>1).sum()

523

### accounts with multiple addresses

In [50]:
account_address_df['account'].value_counts()

bruce-110            267
dutedeyanhuo         151
fmeserver            139
imcda                129
chekou                99
                    ... 
hellominimini          1
helloword20210523      1
helloworld1001231      1
helloworld1001232      1
zzzzzxf                1
Name: account, Length: 12883, dtype: int64

In [ ]:
(account_address_df['account'].value_counts()>1).sum()

1585

### duplicate contributions?  
These contributions can only match one tx, and most of them been created at alomost the same time, maybe they are duplicate records?

In [16]:
df[
    (df['tx_exists_n_times_in_grant'] > 1) & 
    (df['n_account_in_tx'] == 1) & 
    ~df['tx'].isin(df[df['contribution_exists_n_times'] > 1]['tx'].unique())
]

,contribution,account,grant,from,tx,n_account_in_tx,contribution_exists_n_times,tx_exists_n_times_in_grant
6,137250,deadpolaroid,9,0x5da5f4c020f856abdb168fd35c957d6006ba2ede,0xd59184c78fd539e794e6707739d21346396c161a71cc...,1,1,2
7,137251,deadpolaroid,9,0x5da5f4c020f856abdb168fd35c957d6006ba2ede,0xd59184c78fd539e794e6707739d21346396c161a71cc...,1,1,2
1830,135587,contreus,12,0x5eb2cb85bbcdaa9c9802c777a0b996df6b6f26cc,0x447ff9b701b0b98b0d0429f42e900eeffdcfeeea36ad...,1,1,2
1831,135562,contreus,12,0x5eb2cb85bbcdaa9c9802c777a0b996df6b6f26cc,0x447ff9b701b0b98b0d0429f42e900eeffdcfeeea36ad...,1,1,2
1848,156343,quzghun,12,0xdb5fa89c845b180a3dc6c115c65dbcc0f03b70f5,0x245c3887f3718f8105cb17d6618b79520eb1f791b149...,1,1,2
...,...,...,...,...,...,...,...,...
224251,129946,ethacct,2239,0xa2c62a66f6660166838b95db60f234dfb59e765e,0xbeb76f8804ea0e883e817c49306640fac19c18ac544c...,1,1,2
224526,145147,mlibty,2248,0x9eec0b5bd8a48047f0dcc61e98b4b92951480f98,0x255aa7ce561dbb6057d44a688864afe55fc16af32143...,1,1,2
224527,145145,mlibty,2248,0x9eec0b5bd8a48047f0dcc61e98b4b92951480f98,0x255aa7ce561dbb6057d44a688864afe55fc16af32143...,1,1,2
224548,130273,erhenglu,2248,0x0d30ca8dc55d1b5ef9cba4fa504da8341e252cec,0xef1ed021eb2ed1e376dbaa81f1785dae5bed875b3308...,1,1,2
